# Use Case Example

This example illustrates a use case that covers the essential steps involved in building a hydrological model and conducting a climate change analysis:

- **Identification of the watershed and its key characteristics**
  - Beaurivage watershed in Southern Quebec, at the location of the 023401 streamflow gauge.
- **Collection of observed data**
  - ERA5-Land and streamflow gauge data.
- **Preparation and calibration of the hydrological model**
  - GR4JCN emulated by the Raven hydrological framework.
- **Calculation of hydrological indicators**
  - Mean summer flow
  - Mean monthly flow
  - 20- and 100-year maximum flow
  - 2-year minimum 7-day average summer flow
- **Assessment of the impact of climate change**
  - Bias-adjusted CMIP6 simulations from the ESPO-G6-R2 dataset

## Identification of a watershed and its characteristics

<div class="alert alert-info"> <b>INFO</b>

For more information on this section and available options, consult the [GIS module page](https://xhydro.readthedocs.io/en/latest/notebooks/gis.html#GIS-module).

</div>

This first step is highly dependent on the hydrological model. Since we will use GR4JCN in our example, we need to obtain the drainage area, centroid coordinates, and elevation. We'll also need the watershed delineation to extract the meteorological data.

- Delineation of the watershed and basic characteristics: `xhydro.gis.watershed_delineation` and `xhydro.gis.watershed_properties`
- Elevation: `xhydro.gis.surface_properties`

In [ ]:
# Workaround for determining the notebook folder within a running notebook
# This cell is not visible when the documentation is built.

from __future__ import annotations

try:
    from _finder import _find_current_folder

    notebook_folder = _find_current_folder()
except ImportError:
    from pathlib import Path

    notebook_folder = Path().cwd()

(notebook_folder / "_data").mkdir(exist_ok=True)

In [ ]:
import leafmap
import numpy as np
import pandas as pd

import xhydro.gis as xhgis

In [ ]:
# Watershed delineation
coords = (-71.28878, 46.65692)
m = leafmap.Map(center=(coords[1], coords[0]), zoom=8)
gdf = xhgis.watershed_delineation(coordinates=coords, map=m)
gdf

In [ ]:
m

In [ ]:
# Watershed properties
gdf_wat = xhgis.watershed_properties(gdf)
gdf_wat

In [ ]:
# Surface properties
gdf_surf = xhgis.surface_properties(gdf)
gdf_surf

In [ ]:
# Information for GR4JCN
drainage_area = np.array([(gdf_wat["area"] / 1000000).iloc[0]])
latitude = np.array([gdf_wat.centroid[0][1]])
longitude = np.array([gdf_wat.centroid[0][0]])
elevation = np.array([gdf_surf["elevation"].iloc[0]])

Since `xhgis.watershed_delineation` extracts the nearest HydroBASINS polygon, the watershed might not exactly correspond to the requested coordinates. The 023401 streamflow gauge as an associated drainage area of 708 km², which differs from our results. Streamflow will have to be adjusted using an area scaling factor.

In [ ]:
gauge_area = 708
scaling_factor = drainage_area / gauge_area
scaling_factor

## Collection of observed data

In [ ]:
import geopandas as gpd
import hvplot.xarray
import matplotlib.pyplot as plt
import xarray as xr

# For easy access to the specific streamflow data used here
import xdatasets
import xscen

### Meteorological data

<div class="alert alert-info"> <b>INFO</b>

Multiple libraries could be used to perform these steps. For simplicity, this example will use the [`subset`](https://xscen.readthedocs.io/en/latest/notebooks/2_getting_started.html#Defining-the-region) and [`aggregate` modules of the xscen](https://xscen.readthedocs.io/en/latest/notebooks/2_getting_started.html#Spatial-mean) library.

</div>

This example will use daily ERA5-Land data hosted on the PAVICS platform.

In [ ]:
# Extraction of ERA5-Land data
meteo_ref = xr.open_dataset(
    "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/datasets/reanalyses/day_ERA5-Land_NAM.ncml",
    engine="netcdf4",
    chunks={"time": 365, "lon": 50, "lat": 50},
)[["pr", "tasmin", "tasmax"]]
meteo_ref

This dataset covers the entire globe and more has than 70 years of data. The first step is thus to subset the dataset both spatially and temporally. For the spatial subset, the GeoDataFrame obtained earlier can be used.

In [ ]:
meteo_ref = meteo_ref.sel(time=slice("1991", "2020"))  # Temporal subsetting
meteo_ref = xscen.spatial.subset(
    meteo_ref, method="shape", tile_buffer=2, shape=gdf
)  # Spatial subsetting, with a buffer of 2 grid cells
meteo_ref = meteo_ref.assign_coords({"crs": meteo_ref.crs})
meteo_ref

In [ ]:
ax = plt.subplot(1, 1, 1)
meteo_ref.tasmin.isel(time=0).plot(ax=ax)
gdf.plot(ax=ax)

Since GR4JCN is a lumped model, we need to average the meteo over the watershed. This will be accomplished using `xscen.spatial_mean`. Multiple methods exist, but for shapefiles, the most accurate is "xesmf", which calls upon [xESMF's SpatialAverager](https://pangeo-xesmf.readthedocs.io/en/latest/notebooks/Spatial_Averaging.html) to accurately follow each grid cell's representation over the polygon. It can be very slow for detailed polygons with lots of vertices, but the "simplify_tolerance" argument helps speed up that process.

In [ ]:
meteo_ref = xscen.spatial_mean(
    meteo_ref,
    method="xesmf",
    region={"method": "shape", "shape": gdf},
    simplify_tolerance=0.1,
)

# Some housekeeping to associate the HYBAS_ID dimension (coming from the shapefile) to other coordinates added from the GeoDataFrame.
for c in meteo_ref.coords:
    if len(meteo_ref[c].dims) == 0:
        if c not in ["lon", "lat"]:
            meteo_ref = meteo_ref.drop_vars([c])
        else:
            meteo_ref[c] = meteo_ref[c].expand_dims("HYBAS_ID")
meteo_ref

Raven expects temperatures in Celsius and precipitation in millimetres, but they currently are in CF-compliant Kelvin and kg m-2 s-1, respectively. Once again, `xscen` can be used for that operation.

In [ ]:
meteo_ref = xscen.utils.change_units(
    meteo_ref, {"tasmax": "degC", "tasmin": "degC", "pr": "mm"}
)

In [ ]:
# Save (necessary for model calibration)
meteo_ref.to_netcdf(notebook_folder / "_data" / "meteo.nc")

### Hydrometric data

Gauge streamflow data from the Quebec government can be accessed through the `xdatasets` library.

In [ ]:
qobs = (
    xdatasets.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["023401"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1991-01-01", "end": "2020-12-31"},
        }
    )
    .data.squeeze()
    .load()
).sel(spatial_agg="watershed")

In [ ]:
# Once again, some housekeeping is required on the metadata to make sure that xHydro understands the dataset.
qobs["id"].attrs["cf_role"] = "timeseries_id"
qobs = qobs.rename({"id": "station_id"})
qobs["streamflow"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}
for c in qobs.coords:
    if len(qobs[c].dims) == 0 and "time" not in c:
        qobs[c] = qobs[c].expand_dims("station_id")

qobs

In [ ]:
qobs.streamflow.hvplot()

As specified earlier, streamflow observations need to be modified to account for the differences in watershed sizes between the gauge and the polygon.

In [ ]:
with xr.set_options(keep_attrs=True):
    qobs["streamflow"] = qobs["streamflow"] * scaling_factor

In [ ]:
# Save (necessary for model calibration)
qobs.to_netcdf(notebook_folder / "_data" / "qobs.nc")

## Preparation and calibration of the hydrological model (xhydro.modelling)

<div class="alert alert-info"> <b>INFO</b>

For more information on this section and available options, consult the [Hydrological modelling module page](https://xhydro.readthedocs.io/en/latest/notebooks/hydrological_modelling.html).

</div>

In [ ]:
from xhydro.modelling.calibration import perform_calibration
from xhydro.modelling.obj_funcs import get_objective_function

The `perform_calibration` requires a `model_config` argument that allows it to build the corresponding hydrological model. All the required information has been acquired in previous sections, so it is only a matter of filling in the entries of the RavenPy model.

For simplification matters, the meteorological station's elevation will be set as the real watershed's average elevation. Computing grid cell elevation in ERA5-Land is not always trivial and is not within the scope of this example. Similarly, snow water equivalent is not currently available on PAVICS' database, so "AVG_ANNUAL_SNOW" was roughly estimated using [Brown & Brasnett (2010)](https://ccin.ca/ccw/snow/overview/references).

In [ ]:
# Model configuration
model_config = {
    "model_name": "GR4JCN",
    "parameters": [0.529, -3.396, 407.29, 1.072, 16.9, 0.947],
    "drainage_area": drainage_area,
    "elevation": elevation,
    "latitude": latitude,
    "longitude": longitude,
    "start_date": "1991-01-01",
    "end_date": "2020-12-31",
    "qobs_path": notebook_folder / "_data" / "qobs.nc",
    "alt_names_flow": "streamflow",
    "meteo_file": notebook_folder / "_data" / "meteo.nc",
    "data_type": ["TEMP_MAX", "TEMP_MIN", "PRECIP"],
    "alt_names_meteo": {"TEMP_MIN": "tasmin", "TEMP_MAX": "tasmax", "PRECIP": "pr"},
    "meteo_station_properties": {
        "ALL": {"elevation": elevation, "latitude": latitude, "longitude": longitude}
    },
    "rain_snow_fraction": "RAINSNOW_DINGMAN",
    "evaporation": "PET_HARGREAVES_1985",
    "global_parameter": {"AVG_ANNUAL_SNOW": 100.00},
}
model_config["qobs"] = xr.open_dataset(model_config["qobs_path"]).streamflow.values

# Parameter bounds for GR4JCN
bounds_low = [0.01, -15.0, 10.0, 0.0, 1.0, 0.0]
bounds_high = [2.5, 10.0, 700.0, 7.0, 30.0, 1.0]

In [ ]:
# Calibration / validation period
mask_calib = xr.where(qobs.time.dt.year <= 2010, 1, 0).values
mask_valid = xr.where(qobs.time.dt.year > 2010, 1, 0).values

# Model calibration
best_parameters, best_simulation, best_objfun = perform_calibration(
    model_config,
    "kge",
    bounds_low=bounds_low,
    bounds_high=bounds_high,
    evaluations=150,
    algorithm="DDS",
    mask=mask_calib,
    sampler_kwargs=dict(trials=1),
)

The real KGE should be computed from a validation period, using `get_objective_function`.

In [ ]:
get_objective_function(
    qobs=model_config["qobs"],
    qsim=best_simulation,
    obj_func="kge",
    mask=mask_valid,
).values

In [ ]:
ax = plt.figure(figsize=(15, 10))
xr.open_dataset(model_config["qobs_path"]).streamflow.hvplot(
    color="k", line_width=3
) * best_simulation.streamflow.hvplot(color="r")

## Calculation of hydroclimatological indicators

In [ ]:
import xclim

import xhydro as xh
import xhydro.frequency_analysis as xhfa

### Non-frequential indicators

<div class="alert alert-info"> <b>INFO</b>

For more information on this section and available options, consult the [Climate change analysis page](https://xhydro.readthedocs.io/en/latest/notebooks/climate_change.html).

Custom indicators in `xHydro` are built by following the YAML formatting required by `xclim`. More information is available [in the xclim documentation](https://xclim.readthedocs.io/en/latest/api.html#yaml-file-structure). The list of Yaml IDs is available [here](https://xclim.readthedocs.io/en/stable/indicators.html).

</div>

For a climate change impact analysis, the typical process to compute non-frequential indicators would be to:

1. Define the indicators either through the `xclim` functionalities shown below or through a [YAML file](https://xclim.readthedocs.io/en/latest/api.html#yaml-file-structure).
2. Call `xhydro.indicators.compute_indicators`, which would produce annual results through a dictionary, where each key represents the requested frequencies.
3. Call `xhydro.cc.climatological_op` on each entry of the dictionary to compute the 30-year average.
4. Recombine the datasets.

However, if the annual results are not required, `xhydro.cc.produce_horizon` can bypass steps 2 to 4 and alleviate a lot of hassle. It accomplishes that by removing the `time` axis and replacing it for a `horizon` dimension that represents a slice of time. In the case of seasonal or monthly indicators, a corresponding `season` or `month` dimension is also added.

We will compute the mean summer flow (an annual indicator) and mean monthly flows.

In [ ]:
indicators = [
    # Mean summer flow
    xclim.core.indicator.Indicator.from_dict(
        data={
            "base": "stats",
            "input": {"da": "streamflow"},
            "parameters": {"op": "mean", "indexer": {"month": [6, 7, 8]}},
        },
        identifier="qmoy_summer",
        module="hydro",
    ),
    # Mean monthly flow
    xclim.core.indicator.Indicator.from_dict(
        data={
            "base": "stats",
            "input": {"da": "streamflow"},
            "parameters": {"op": "mean", "freq": "MS"},
        },
        identifier="qmoy_monthly",
        module="hydro",
    ),
]

ds_indicators = xh.cc.produce_horizon(
    best_simulation, indicators=indicators, periods=["1991", "2020"]
)

In [ ]:
ds_indicators

### Frequency analysis

<div class="alert alert-info"> <b>INFO</b>

For more information on this section and available options, consult the [Local frequency analysis page](https://xhydro.readthedocs.io/en/latest/notebooks/local_frequency_analysis.html).

</div>

A frequency analysis typically follows these steps:

1. Get the raw data needed for the analysis, such as annual maximums, through `xhydro.indicators.get_yearly_op`.
2. Call `xhfa.local.fit` to obtain the parameters for a specified number of distributions, such as Gumbel, GEV, and Pearson-III.
3. (Optional) Call `xhfa.local.criteria` to obtain goodness-of-fit parameters.
4. Call `xhfa.local.parametric_quantiles` to obtain the return levels.

We will compute the 20 and 100-year annual maximums, as well as the 2-year minimum 7-day summer flow.

In [ ]:
qref_max = xh.indicators.get_yearly_op(
    best_simulation,
    op="max",
    timeargs={"annual": {}},
    missing="pct",
    missing_options={"tolerance": 0.15},
)
qref_min = xh.indicators.get_yearly_op(
    best_simulation,
    op="min",
    window=7,
    timeargs={"summer": {"date_bounds": ["05-01", "11-30"]}},
    missing="pct",
    missing_options={"tolerance": 0.15},
)

In [ ]:
ax = plt.figure(figsize=(15, 10))
qref_max.streamflow_max_annual.dropna(
    "time", how="all"
).hvplot() * qref_min.streamflow7_min_summer.dropna("time", how="all").hvplot()

In [ ]:
# Frequency analysis applied on multiple distributions
params_max = xhfa.local.fit(
    qref_max,
    distributions=["genextreme", "gumbel_r", "norm", "pearson3"],
    method="MLE",
    min_years=20,
).compute()
params_min = xhfa.local.fit(
    qref_min,
    distributions=["genextreme", "gumbel_r", "norm", "pearson3"],
    method="MLE",
    min_years=20,
).compute()

params_max

While not foolproof, the best fit can be identified using the Bayesian Information Criteria.

In [ ]:
criteria_max = xhfa.local.criteria(qref_max, params_max)
criteria_max = str(
    criteria_max.isel(
        scipy_dist=criteria_max.streamflow_max_annual.sel(criterion="bic")
        .argmin()
        .values
    ).scipy_dist.values
)  # Get the best fit as a string
print(f"Best distribution for the annual maxima: {criteria_max}")

criteria_min = xhfa.local.criteria(qref_min, params_min)
criteria_min = str(
    criteria_min.isel(
        scipy_dist=criteria_min.streamflow7_min_summer.sel(criterion="bic")
        .argmin()
        .values
    ).scipy_dist.values
)  # Get the best fit as a string
print(f"Best distribution for the summer 7-day minima: {criteria_min}")

In [ ]:
# Plotting
data_max = xhfa.local._prepare_plots(
    params_max, xmin=1, xmax=1000, npoints=50, log=True
)
data_max["streamflow_max_annual"].attrs["long_name"] = "streamflow"
pp = xhfa.local._get_plotting_positions(qref_max[["streamflow_max_annual"]])

data_max.streamflow_max_annual.sel(scipy_dist=criteria_max).hvplot(
    x="return_period", grid=True, groupby=[], logx=True, color="k"
) * pp.hvplot.scatter(
    x="streamflow_max_annual_pp",
    y="streamflow_max_annual",
    grid=True,
    groupby=[],
    logx=True,
    color="k",
)

In [ ]:
# Plotting
data_min = xhfa.local._prepare_plots(
    params_min, xmin=1, xmax=1000, npoints=50, log=True
)
data_min["streamflow7_min_summer"].attrs["long_name"] = "streamflow"
pp = xhfa.local._get_plotting_positions(qref_min[["streamflow7_min_summer"]])

data_min.streamflow7_min_summer.sel(scipy_dist=criteria_min).hvplot(
    x="return_period", grid=True, groupby=[], logx=True, color="k"
) * pp.hvplot.scatter(
    x="streamflow7_min_summer_pp",
    y="streamflow7_min_summer",
    grid=True,
    groupby=[],
    logx=True,
    color="k",
)

In [ ]:
# Computation of return levels
rl_max = xhfa.local.parametric_quantiles(
    params_max.sel(scipy_dist=criteria_max).expand_dims("scipy_dist"), t=[20, 100]
).squeeze()
rl_min = xhfa.local.parametric_quantiles(
    params_min.sel(scipy_dist=criteria_min).expand_dims("scipy_dist"), t=[2], mode="min"
).squeeze()
rl_max

In [ ]:
print(
    f"20-year annual maximum: {np.round(rl_max.streamflow_max_annual.sel(return_period=20).values, 1)} m³/s"
)
print(
    f"100-year annual maximum: {np.round(rl_max.streamflow_max_annual.sel(return_period=100).values, 1)} m³/s"
)
print(
    f"2-year minimum 7-day summer flow: {np.round(rl_min.streamflow7_min_summer.values, 1)} m³/s"
)

## Future streamflow simulations

### Future meteorological data

Now that we have access to a calibrated hydrological model and historical indicators, we can perform the climate change analysis. This example will use a set of CMIP6 models that have been bias adjusted using ERA5-Land, for consistency with the reference product. Specifically, we will use the ESPO-G6-R2 dataset, also hosted on PAVICS. While it is recommended to use multiple emission scenarios, this example will only use the SSP2-4.5 simulations from 14 climate models.

We can mostly reuse the same code as above. One difference is that climate models often use custom calendars that need to be converted back to standard ones. Once again, `xscen` can be used for that.

In [ ]:
models = [
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_CAS_FGOALS-g3_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_CMCC_CMCC-ESM2_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_CSIRO-ARCCSS_ACCESS-CM2_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_CSIRO_ACCESS-ESM1-5_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_DKRZ_MPI-ESM1-2-HR_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_INM_INM-CM5-0_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_MIROC_MIROC6_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_MPI-M_MPI-ESM1-2-LR_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_MRI_MRI-ESM2-0_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_NCC_NorESM2-LM_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_CNRM-CERFACS_CNRM-ESM2-1_ssp245_r1i1p1f2_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_NIMS-KMA_KACE-1-0-G_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_NOAA-GFDL_GFDL-ESM4_ssp245_r1i1p1f1_1950-2100.ncml",
    "day_ESPO-G6-E5L_v1.0.0_CMIP6_ScenarioMIP_NAM_BCC_BCC-CSM2-MR_ssp245_r1i1p1f1_1950-2100.ncml",
]

In [ ]:
def _prepare_climate(model):
    meteo_sim = xr.open_dataset(
        f"https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/datasets/simulations/bias_adjusted/cmip6/ouranos/ESPO-G/ESPO-G6-E5Lv1.0.0/{model}",
        engine="netcdf4",
        chunks={"time": 365, "lon": 50, "lat": 50},
    )
    meteo_sim = xscen.spatial.subset(
        meteo_sim, method="shape", tile_buffer=2, shape=gdf
    )
    meteo_sim = meteo_sim.assign_coords({"crs": meteo_sim.crs})
    meteo_sim = xscen.spatial_mean(
        meteo_sim,
        method="xesmf",
        region={"method": "shape", "shape": gdf},
        simplify_tolerance=0.1,
    )
    for c in meteo_sim.coords:
        if len(meteo_sim[c].dims) == 0:
            if c not in ["lon", "lat"]:
                meteo_sim = meteo_sim.drop_vars([c])
            else:
                meteo_sim[c] = meteo_sim[c].expand_dims("HYBAS_ID")

    meteo_sim = xscen.utils.change_units(
        meteo_sim, {"tasmax": "degC", "tasmin": "degC", "pr": "mm"}
    )

    # Manage calendars
    meteo_sim = xscen.utils.clean_up(
        meteo_sim,
        convert_calendar_kwargs={"calendar": "standard", "align_on": "random"},
        missing_by_var={"tasmin": "interpolate", "tasmax": "interpolate", "pr": 0},
    )

    return meteo_sim


for i, model in enumerate(models):
    out = _prepare_climate(model)
    out.to_netcdf(notebook_folder / "_data" / f"meteo_sim{i}.nc")

In [ ]:
out.pr.hvplot()

### Future streamflow data

Once again, the same code as before can be roughly reused here, but with `xhydro.modelling.hydrological_model`. The main difference is that the best parameters can be used when setting up the hydrological model, and that the dates are the full range going to 2100.

In [ ]:
from copy import deepcopy

import xhydro.modelling as xhm

In [ ]:
model_config = {
    "model_name": "GR4JCN",
    "parameters": np.array(best_parameters),
    "drainage_area": drainage_area,
    "elevation": elevation,
    "latitude": latitude,
    "longitude": longitude,
    "start_date": "1991-01-01",
    "end_date": "2099-12-31",
    "qobs_path": notebook_folder / "_data" / "qobs.nc",
    "alt_names_flow": "streamflow",
    "meteo_file": "meteo_sim.nc",
    "data_type": ["TEMP_MAX", "TEMP_MIN", "PRECIP"],
    "alt_names_meteo": {"TEMP_MIN": "tasmin", "TEMP_MAX": "tasmax", "PRECIP": "pr"},
    "meteo_station_properties": {
        "ALL": {"elevation": elevation, "latitude": latitude, "longitude": longitude}
    },
    "rain_snow_fraction": "RAINSNOW_DINGMAN",
    "evaporation": "PET_HARGREAVES_1985",
    "global_parameter": {"AVG_ANNUAL_SNOW": 100.00},
}

In [ ]:
# Model execution
for i in range(len(models)):
    model_cfg = deepcopy(model_config)
    model_cfg["meteo_file"] = notebook_folder / "_data" / f"meteo_sim{i}.nc"
    qsim = xhm.hydrological_model(model_cfg).run()
    qsim.to_netcdf(notebook_folder / "_data" / f"qsim{i}.nc")

In [ ]:
qsim.streamflow.hvplot()